## Colorado River Water Rights from Utah Division of Water Rights

### Import data from Utah Division of Water Rights website (per County)

In [1]:
import requests

URL = "https://www.waterrights.utah.gov/distinfo/colorado/WRPriorityDDview.asp?county=CN"
page = requests.get(URL)

In [2]:
from bs4 import BeautifulSoup, Tag

soup = BeautifulSoup(page.content, "html.parser")

In [3]:
# Find current run date and last updated date
print(soup.find("h2", class_=False, id=False).find_next_sibling(text=True))


 

Program Updated: February 5, 2021

      
Run Date: August 20, 2022



Create dataframe `df` with each water right for specified county

In [4]:
table = soup.findAll("table")[0].find("table") # finds table inside table
# print(table)

import pandas as pd
import numpy as np
import re

# Defining of the dataframe
df = pd.DataFrame(columns=['RowId', 
                           'Water Right', 
                           'Priority Year', 
                           'Priority Month', 
                           'Priority Day', 
                           'Diversion (acft)',
                           'Depletion (acft)',
                           'Cumulative Diversion (acft)',
                           'Cumulative Depletion (acft)',
                           'County',
                           'Source',
                           'Status',
                           'Owner Name'
                          ])
df.head()

# Collecting data
for row in table.find_all('tr')[2:]:  #skip first two rows which are headers   
    # Find all data for each column
    columns = row.find_all('td')
    
    row_id = columns[0].text
    water_right = columns[1].text
    year = columns[2].get_text(strip=True)
    month = columns[3].get_text(strip=True)
    day = columns[4].get_text(strip=True)
    diversion = columns[5].text
    depletion = columns[6].text
    cum_diversion = columns[7].text
    cum_depletion = columns[8].text
    county = columns[9].text
    source = columns[10].text
    status = columns[11].text
    
    if columns[12].find('br'):
        ls = columns[12].contents
        for i in ls:
            if isinstance(i, Tag):
                i.extract()
        owner = columns[12].contents
    else:
        owner = columns[12].text

    df = df.append({
        'RowId': row_id,  
        'Water Right': water_right, 
        'Priority Year': year, 
        'Priority Month': month, 
        'Priority Day': day, 
        'Diversion (acft)': diversion,
        'Depletion (acft)': depletion,
        'Cumulative Diversion (acft)': cum_diversion,
        'Cumulative Depletion (acft)': cum_depletion,
        'County': county,
        'Source': source,
        'Status': status,
        'Owner Name': owner
        }, ignore_index=True)
    



In [5]:
print(df.head())
print(df.size) #52624

  RowId Water Right Priority Year Priority Month Priority Day  \
0     1     91-1715                                             
1     2     91-5208                                             
2     3     91-5217                                             
3     4     91-5224                                             
4     5     91-5229                                             

  Diversion (acft) Depletion (acft) Cumulative Diversion (acft)  \
0            87.53            47.53                       87.53   
1             0.60             0.26                       88.13   
2             0.82             0.27                       88.95   
3             0.60             0.26                       89.55   
4             0.60             0.26                       90.15   

  Cumulative Depletion (acft)                          County  \
0                       47.53  Carbon                           
1                       47.79  Carbon                           
2          

If multiple owners per water right detected, write to separate rows of `df`

In [6]:
for i in range(len(df)):
    if type(df.loc[i,'Owner Name']) == list:
#         print(df.loc[i,'Owner Name'])
        for elem in df.loc[i,'Owner Name']:
            df = df.append({
                    'RowId': df.loc[i,'RowId'],  
                    'Water Right': df.loc[i,'Water Right'], 
                    'Priority Year': df.loc[i,'Priority Year'], 
                    'Priority Month': df.loc[i,'Priority Month'], 
                    'Priority Day': df.loc[i,'Priority Day'], 
                    'Diversion (acft)': df.loc[i,'Diversion (acft)'],
                    'Depletion (acft)': df.loc[i,'Depletion (acft)'],
                    'Cumulative Diversion (acft)': df.loc[i,'Cumulative Diversion (acft)'],
                    'Cumulative Depletion (acft)': df.loc[i,'Cumulative Depletion (acft)'],
                    'County': df.loc[i,'County'],
                    'Source': df.loc[i,'Source'],
                    'Status': df.loc[i,'Status'],
                    'Owner Name': elem
                    }, ignore_index=True)



Remove any duplicates

In [7]:
for i in range(len(df)):
    if type(df.loc[i,'Owner Name']) == list:
        df.drop([i], axis=0, inplace=True)

df.drop_duplicates()

,RowId,Water Right,Priority Year,Priority Month,Priority Day,Diversion (acft),Depletion (acft),Cumulative Diversion (acft),Cumulative Depletion (acft),County,Source,Status,Owner Name
31,32,91-312,1860,,,0.06,0.06,92.61,49.66,Carbon,Unnamed Spring ...,P,Pete Jr. Stamatakis
32,33,91-350,1860,,,0.12,0.12,92.73,49.78,Carbon,Unnamed Spring ...,P,Dean C. Shimmin
33,34,91-352,1860,,,3.05,2.69,95.78,52.47,Carbon,Hopkins Creek ...,P,Clarence Anderson
35,36,91-423,1860,,,0.00,0.00,97.28,53.97,Carbon,Winter Quarters Creek ...,P,"Liodakis Ranch, LLC"
36,37,91-425,1860,,,0.00,0.00,97.28,53.97,Carbon,Unnamed Spring ...,P,"Liodakis Ranch, LLC"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5558,4031,90-1854,2008,04,18,0.93,2.62,"230,110.33","111,616.73",Carbon,Underground Water Well ...,P,Kenneth Valdez
5559,4033,91-5171,2010,10,28,4.18,2.69,"230,120.21","111,622.44",Carbon,Ponds ...,P,James Allen Staker
5560,4033,91-5171,2010,10,28,4.18,2.69,"230,120.21","111,622.44",Carbon,Ponds ...,P,USA Bureau of Land Management
5561,4044,91-5266,2020,07,21,0.00,0.00,"230,152.29","111,650.59",Carbon,Underground Water Well ...,A,Josh Hauser


Make a copy of the dataframe before any additional data cleaning

In [8]:
df_raw_scraped = df

## Data Cleaning:

* Update column types to numeric, etc. as applicable
* Create full date column
* Handle missing values
* Remove commas in numeric columns
* trim leading and trailing whitespace
* Replace & with and
* Remove any punctuation before grouping owner names

Remove commas from Depletion and Diversion-related fields

In [9]:
df['Depletion (acft)'] = df['Depletion (acft)'].replace(',','', regex=True)
df['Diversion (acft)'] = df['Diversion (acft)'].replace(',','', regex=True)
df['Cumulative Depletion (acft)'] = df['Cumulative Depletion (acft)'].replace(',','', regex=True)
df['Cumulative Diversion (acft)'] = df['Cumulative Diversion (acft)'].replace(',','', regex=True)

Convert specified columns to numeric data types

In [10]:
c = ['RowId', 'Diversion (acft)', 'Depletion (acft)', 'Cumulative Depletion (acft)', 'Cumulative Diversion (acft)']
df[c] = df[c].apply(pd.to_numeric,errors='coerce')

In [11]:
df = df.replace(r'', np.nan, regex=True)
df['Owner Name'] = df['Owner Name'].str.strip().str.replace('&', 'and').str.replace('[^\w\s]','')

In [12]:
df.loc[df['Priority Year'].notnull() & df['Priority Month'].notnull() & df['Priority Day'].notnull(), 'Priority Date'] = df['Priority Year'] + '-' + df['Priority Month'] + '-' + df['Priority Day']
df.loc[df['Priority Year'].notnull() & df['Priority Month'].isnull() & df['Priority Day'].isnull(), 'Priority Date'] = df['Priority Year']
df.loc[df['Priority Year'].notnull() & df['Priority Month'].notnull() & df['Priority Day'].isnull(), 'Priority Date'] = df['Priority Year'] + '-' + df['Priority Month']

In [15]:
# Add water rights URL to dataframe
water_right_base_url = 'https://www.waterrights.utah.gov/asp_apps/wrprint/wrprint.asp?wrnum='
df['Water Right URL'] = water_right_base_url + df['Water Right'].astype(str)

df = df[['County', 'RowId', 'Water Right', 'Water Right URL', 'Priority Date', 'Diversion (acft)', 'Depletion (acft)', 'Cumulative Diversion (acft)', 'Cumulative Depletion (acft)', 'Source', 'Status', 'Owner Name']].sort_values(by=['RowId', 'Diversion (acft)', 'Depletion (acft)'],ignore_index=True)



In [16]:
df

,County,RowId,Water Right,Water Right URL,Priority Date,Diversion (acft),Depletion (acft),Cumulative Diversion (acft),Cumulative Depletion (acft),Source,Status,Owner Name
0,Carbon,1,91-1715,https://www.waterrights.utah.gov/asp_apps/wrpr...,NaN,87.53,47.53,87.53,47.53,Horse Canyon Creek ...,P,Jensen Ranches LLC
1,Carbon,1,91-1715,https://www.waterrights.utah.gov/asp_apps/wrpr...,NaN,87.53,47.53,87.53,47.53,Horse Canyon Creek ...,P,Security Title Company
2,Carbon,2,91-5208,https://www.waterrights.utah.gov/asp_apps/wrpr...,NaN,0.60,0.26,88.13,47.79,Price River ...,P,Earl Koskie
3,Carbon,2,91-5208,https://www.waterrights.utah.gov/asp_apps/wrpr...,NaN,0.60,0.26,88.13,47.79,Price River ...,P,Price River Water Users Assn
4,Carbon,2,91-5208,https://www.waterrights.utah.gov/asp_apps/wrpr...,NaN,0.60,0.26,88.13,47.79,Price River ...,P,Mark Sullivan
...,...,...,...,...,...,...,...,...,...,...,...,...
5089,Carbon,4044,91-5266,https://www.waterrights.utah.gov/asp_apps/wrpr...,2020-07-21,0.00,0.00,230152.29,111650.59,Underground Water Well ...,A,Kristin Hauser
5090,Carbon,4045,91-239,https://www.waterrights.utah.gov/asp_apps/wrpr...,2021-02-10,3500.00,3500.00,233652.29,115150.59,Range Creek ...,A,City of Sunnyside
5091,Carbon,4046,90-1893,https://www.waterrights.utah.gov/asp_apps/wrpr...,2021-04-27,0.00,0.00,233652.29,115150.59,Unnamed Spring ...,A,James H II and Jodi R Lantz
5092,Carbon,4047,90-1895,https://www.waterrights.utah.gov/asp_apps/wrpr...,2022-02-01,0.00,0.00,233652.29,115150.59,Unnamed Spring ...,A,Range Creek Properties LLC


In [17]:
df['Total Diversion Rights'] = df.groupby('Owner Name')['Diversion (acft)'].transform('sum').round(1)
df['Total Depletion Rights'] = df.groupby('Owner Name')['Depletion (acft)'].transform('sum').round(1)
df['# of Water Rights'] = df.groupby('Owner Name')['Water Right'].transform('nunique')

In [18]:
county_names = df[['Owner Name']].sort_values(by = 'Owner Name').drop_duplicates()
summary_df = df[['Owner Name', 'Total Depletion Rights', 'Total Diversion Rights', '# of Water Rights']].sort_values(by = ['Total Depletion Rights'], ascending=False).drop_duplicates()

In [19]:
summary_df.to_csv('Carbon_County_Summary.csv', index=False)
county_names.to_csv('Carbon_County_OwnerNames.csv', index=False)

## Fuzzy Match Similar Owners to Single Owner

1. Find closest fuzzy matched names for each name in list
2. Find Owner Names that have more than 1 name in their list (name other than their own) with a fuzzy match > 85%
3. Manually verify fuzzy match (remove if the names in list do not actually match)
4. Create new dictionary with the alternative Owner Name that should be used for each field (or itself if only one match)
5. Replace the original Owner Names with their alternative names from the new dictionary

In [20]:
from fuzzywuzzy import fuzz

alt_county_names = pd.DataFrame(columns=['Owner Name', 'Alternative Owner Name', 'Fuzzy Ratio'])

for index, row in county_names.iterrows():
    for index2, row2 in county_names[county_names['Owner Name'] != row['Owner Name']].iterrows():
        if fuzz.ratio(row['Owner Name'], row2['Owner Name']) > 85:
            
            print(row['Owner Name'], ',', row2['Owner Name'], ':', fuzz.ratio(row['Owner Name'], row2['Owner Name']))

            alt_county_names = alt_county_names.append({
                'Owner Name': row['Owner Name'],
                'Alternative Owner Name': row2['Owner Name'],
                'Fuzzy Ratio': fuzz.ratio(row['Owner Name'], row2['Owner Name'])
            }, ignore_index=True)
            
    print('-----------------------------')


/Users/remolloy101/opt/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
----------

-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
E E Peirce , EE Peirce : 95
-----------------------------
-----------------------------
-----------------------------
EE Peirce , E E Peirce : 95
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
Elaine C Conover , Elaine Conover : 93
-----------------------------
Elaine Conover , Elaine C Conover : 93
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
--------------------------

-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
John D and Cherie Critchlow , Judson D and Cherie Critchlow : 93
John D and Cherie Critchlow , Judson Dorse and Cherie Critchlow : 87
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
John R D Ambrosio , John RD Ambrosio : 97
--------------

-----------------------------
Pacificorp dba Utah Power and Light Company , PacifiCorp dba Utah Power and Light Company : 98
Pacificorp dba Utah Power and Light Company , PacifiCorp dba Utah Power and Light company : 95
-----------------------------
Paradise Mountain No 1 LC , Paradise Mountain No 2 LC : 96
-----------------------------
Paradise Mountain No 2 LC , Paradise Mountain No 1 LC : 96
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
Pete Frandsen , G Pete Frandsen : 93
-----------------------------
Pete Jr Stamatakis , Pete Stamatakis : 91
-----------------------------
Pete Stamatakis , Pete Jr Stamatakis : 91
-----------------------------
-----------------------------
-

-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
-----------------------------
Utah School and Institutional Trust Lands Admin , School and Institutional Trust Lands Administration : 86
Utah School and Institutional Trust Lands Admin , State of Utah School and Institutional Trust Lands Admin : 91
Utah School and Institutional Trust Lands Admin , Utah School and Institutional Trust Lands Administration : 91
-----------------------------
Utah School and Institutional Trust Lands Administration , School and Institutional Trust Lands Administration : 95
Utah School and Institutional Trust Lands Administration , Utah School and Institutional Trust Lands Admin : 91
-----------------------------
-----------------------------
-----------------------------
---------------------

Create a dictionary with all unique owner names and potential alternative owner names:

In [21]:
alt_owner_names_dict = {}

for index, row in county_names.iterrows():
    
    alt_owner_names_dict[row['Owner Name']] = [row['Owner Name']]

    for index2, row2 in county_names[county_names['Owner Name'] != row['Owner Name']].iterrows():
        if fuzz.ratio(row['Owner Name'], row2['Owner Name']) > 85:
            
            alt_owner_names_dict[row['Owner Name']].append(row2['Owner Name'])

            
print('Dictionary Complete!')

Dictionary Complete!


In [ ]:
alt_owner_names_dict

Sort list of values for each key

In [22]:
alt_owner_names_dict_sorted = dict()
for key in sorted(alt_owner_names_dict):
    alt_owner_names_dict_sorted[key] = sorted(alt_owner_names_dict[key])

alt_owner_names_dict_sorted

{'2BMP LLC': ['2BMP LLC'],
 '5L Enterprises': ['5L Enterprises'],
 '623 Canyon Asset LLC': ['623 Canyon Asset LLC'],
 'A Dan and Jacquelyn B Oman': ['A Dan and Jacquelyn B Oman'],
 'A Gale Jorgensen': ['A Gale Jorgensen'],
 'AJB Holdings LLC': ['AJB Holdings LLC'],
 'ANR Co Inc': ['ANR Co Inc'],
 'ARMS LLC': ['ARMS LLC'],
 'ATL5 LLC': ['ATL5 LLC'],
 'Aaron E and Deborah Sorensen': ['Aaron E and Deborah Sorensen'],
 'Agee Real Estate Holdings LLC': ['Agee Real Estate Holdings LLC'],
 'Alan Paul Riddle': ['Alan Paul Riddle'],
 'Albert B and April S Cornaby': ['Albert B and April S Cornaby'],
 'Albert J and Shirley A Spensko': ['Albert J and Shirley A Spensko'],
 'Albert Thayn': ['Albert Thayn'],
 'Albert and Leona Leautaud Trust': ['Albert and Leona Leautaud Trust'],
 'Alfred and Jacquelyn Oman Family Living Trust': ['Alfred and Jacquelyn Oman Family Living Trust'],
 'Alice Jensen': ['Alice Jensen'],
 'Allred Ditch Company': ['Allred Ditch Company'],
 'Alma Powell': ['Alma Powell'],
 'Al

In [ ]:
length_dict = {key: len(value) for key, value in alt_owner_names_dict.items()}

# Output only Owner Names where more at least 1 alternative name found
owner_names_w_alts = { k: v for k, v in length_dict.items() if v > 1 }
list_owner_names_to_remove = ['Pioneer Canal Company No 1', 
                          'Pioneer Canal Company No 2', 
                          'Ellen R Radakovich Marital and Family Trust',
                         'Robert Radakovich Marital and Family Trust',
                         'Sanpete Water Conservancy District',
                         'Carbon Water Conservancy District',
                         'John D and Cherie Critchlow',
                         'Jay L Critchlow',
                         'J D Critchlow',
                         'Enervest Energy Institutional Fund XIIIA LP',
                         'Enervest Energy Institutional Fund XIIIWIB LP',
                         'Enervest Energy Institutional Fund XIIIWIC LP'
                             ]
[owner_names_w_alts.pop(key) for key in list_owner_names_to_remove]

owner_names_w_alts

### Manually change alternative Owner Name to use in dictionary

Take 1st name in list of alternative Owner Names as Alternative Owner Name to replace original dataframe with

In [23]:
alt_names_df_ex = pd.DataFrame.from_dict(alt_owner_names_dict_sorted.items())
alt_names_df_ex.columns = ['Owner Name', 'Alternative Owner Name']
alt_names_df_ex['Alternative Owner Name']
alt_names_df_ex['Alternative Owner Name'] = alt_names_df_ex['Alternative Owner Name'].apply(lambda x: x[0])
alt_names_df_ex[alt_names_df_ex['Owner Name'] == 'Gilmer and Doroth J Nielsen']
alt_names_df_ex[alt_names_df_ex['Owner Name'] == 'Gilmer and Dorothy Nielsen']
alt_names_df_ex[alt_names_df_ex['Owner Name'] == 'Gilmer and doroth J Nielsen']

,Owner Name,Alternative Owner Name
313,Gilmer and doroth J Nielsen,Gilmer and Doroth J Nielsen


Manually check any that Owner Names that are not true matches and remove from dictionary of list of Owner Names that have at least 1 alternative owner name

In [24]:
list_owner_names_to_remove = ['Pioneer Canal Company No 1', 
                          'Pioneer Canal Company No 2', 
                          'Ellen R Radakovich Marital and Family Trust',
                         'Robert Radakovich Marital and Family Trust',
                         'Sanpete Water Conservancy District',
                         'Carbon Water Conservancy District',
                         'John D and Cherie Critchlow',
                         'Jay L Critchlow',
                         'J D Critchlow',
                         'Enervest Energy Institutional Fund XIIIA LP',
                         'Enervest Energy Institutional Fund XIIIWIB LP',
                         'Enervest Energy Institutional Fund XIIIWIC LP'
                             ]

In [25]:
for index, row in alt_names_df_ex.iterrows():
    if row['Owner Name'] in list_owner_names_to_remove:
        print(row['Owner Name'])
        alt_names_df_ex['Alternative Owner Name'][index] = row['Owner Name']
        print(row[['Owner Name', 'Alternative Owner Name']])
        print('--------------------')
alt_names_df_ex

Carbon Water Conservancy District
Owner Name                Carbon Water Conservancy District
Alternative Owner Name    Carbon Water Conservancy District
Name: 111, dtype: object
--------------------
Ellen R Radakovich Marital and Family Trust
Owner Name                Ellen R Radakovich Marital and Family Trust
Alternative Owner Name    Ellen R Radakovich Marital and Family Trust
Name: 240, dtype: object
--------------------
Enervest Energy Institutional Fund XIIIA LP
Owner Name                Enervest Energy Institutional Fund XIIIA LP
Alternative Owner Name    Enervest Energy Institutional Fund XIIIA LP
Name: 245, dtype: object
--------------------
Enervest Energy Institutional Fund XIIIWIB LP
Owner Name                Enervest Energy Institutional Fund XIIIWIB LP
Alternative Owner Name    Enervest Energy Institutional Fund XIIIWIB LP
Name: 246, dtype: object
--------------------
Enervest Energy Institutional Fund XIIIWIC LP
Owner Name                Enervest Energy Institutional Fu

,Owner Name,Alternative Owner Name
0,2BMP LLC,2BMP LLC
1,5L Enterprises,5L Enterprises
2,623 Canyon Asset LLC,623 Canyon Asset LLC
3,A Dan and Jacquelyn B Oman,A Dan and Jacquelyn B Oman
4,A Gale Jorgensen,A Gale Jorgensen
...,...,...
826,William W and Faye F Branson,William W and Faye F Branson
827,William and Armida Sherman,William and Armida Sherman
828,William and Carolyn Barney,William and Carolyn Barney
829,Willis and Wilma Hammerschmid,Willis and Wilma Hammerschmid


In [ ]:
df

In [26]:
df_names_fixed = df.merge(alt_names_df_ex, how='left', left_on = 'Owner Name', right_on = 'Owner Name')[['RowId', 'Water Right', 'Water Right URL','Priority Date', 'Diversion (acft)','Depletion (acft)','Cumulative Diversion (acft)','Cumulative Depletion (acft)','County','Source', 'Status', 'Alternative Owner Name', 'Total Diversion Rights', 'Total Depletion Rights', '# of Water Rights']]
df_names_fixed = df_names_fixed.rename(columns={'Alternative Owner Name':'Owner Name'})
df_names_fixed



,RowId,Water Right,Water Right URL,Priority Date,Diversion (acft),Depletion (acft),Cumulative Diversion (acft),Cumulative Depletion (acft),County,Source,Status,Owner Name,Total Diversion Rights,Total Depletion Rights,# of Water Rights
0,1,91-1715,https://www.waterrights.utah.gov/asp_apps/wrpr...,NaN,87.53,47.53,87.53,47.53,Carbon,Horse Canyon Creek ...,P,Jensen Ranches LLC,177.0,97.0,4
1,1,91-1715,https://www.waterrights.utah.gov/asp_apps/wrpr...,NaN,87.53,47.53,87.53,47.53,Carbon,Horse Canyon Creek ...,P,Security Title Company,192.2,112.2,20
2,2,91-5208,https://www.waterrights.utah.gov/asp_apps/wrpr...,NaN,0.60,0.26,88.13,47.79,Carbon,Price River ...,P,Earl Koskie,1.1,0.7,2
3,2,91-5208,https://www.waterrights.utah.gov/asp_apps/wrpr...,NaN,0.60,0.26,88.13,47.79,Carbon,Price River ...,P,Price River Water Users Assn,5.0,2.1,17
4,2,91-5208,https://www.waterrights.utah.gov/asp_apps/wrpr...,NaN,0.60,0.26,88.13,47.79,Carbon,Price River ...,P,Mark Sullivan,1.1,0.7,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5089,4044,91-5266,https://www.waterrights.utah.gov/asp_apps/wrpr...,2020-07-21,0.00,0.00,230152.29,111650.59,Carbon,Underground Water Well ...,A,Kristin Hauser,0.0,0.0,1
5090,4045,91-239,https://www.waterrights.utah.gov/asp_apps/wrpr...,2021-02-10,3500.00,3500.00,233652.29,115150.59,Carbon,Range Creek ...,A,City of Sunnyside,3500.0,3500.0,1
5091,4046,90-1893,https://www.waterrights.utah.gov/asp_apps/wrpr...,2021-04-27,0.00,0.00,233652.29,115150.59,Carbon,Unnamed Spring ...,A,James H II and Jodi R Lantz,0.0,0.0,1
5092,4047,90-1895,https://www.waterrights.utah.gov/asp_apps/wrpr...,2022-02-01,0.00,0.00,233652.29,115150.59,Carbon,Unnamed Spring ...,A,Range Creek Properties LLC,0.0,0.0,1


In [28]:
df_names_fixed[df_names_fixed['Owner Name'] == 'Gilmer and doroth J Nielsen']

,RowId,Water Right,Water Right URL,Priority Date,Diversion (acft),Depletion (acft),Cumulative Diversion (acft),Cumulative Depletion (acft),County,Source,Status,Owner Name,Total Diversion Rights,Total Depletion Rights,# of Water Rights


In [29]:
alt_names_df_ex[alt_names_df_ex['Owner Name'] == 'Gilmer and Doroth J Nielsen']
alt_names_df_ex[alt_names_df_ex['Owner Name'] == 'Gilmer and Dorothy Nielsen']
alt_names_df_ex[alt_names_df_ex['Owner Name'] == 'Gilmer and doroth J Nielsen']

,Owner Name,Alternative Owner Name
313,Gilmer and doroth J Nielsen,Gilmer and Doroth J Nielsen


In [33]:
df_names_fixed['Total Diversion Rights'] = df_names_fixed.groupby('Owner Name')['Diversion (acft)'].transform('sum').round(1)
df_names_fixed['Total Depletion Rights'] = df_names_fixed.groupby('Owner Name')['Depletion (acft)'].transform('sum').round(1)
df_names_fixed['# of Water Rights'] = df_names_fixed.groupby('Owner Name')['Water Right'].transform('nunique')

df_names_fixed[['Owner Name', 'County', 'Total Diversion Rights', 'Total Depletion Rights', '# of Water Rights']].sort_values(by='Total Depletion Rights', ascending=False).drop_duplicates()


,Owner Name,County,Total Diversion Rights,Total Depletion Rights,# of Water Rights
4555,Price River Water Users Assn,Carbon,54713.5,31126.5,73
2451,PacifiCorp dba Utah Power and Light Company,Carbon,21629.2,17375.6,9
4820,East Carbon City,Carbon,16199.4,10083.8,21
2445,Wellington Canal Company,Carbon,18255.5,9312.3,7
4576,Carbon Canal Company,Carbon,11312.5,6859.5,12
...,...,...,...,...,...
61,Robert Baird,Carbon,0.0,0.0,1
63,Matthew and Shelby Lund,Carbon,0.0,0.0,1
66,Shon Rindlisbacher,Carbon,0.0,0.0,1
68,Slavic Evangelical Baptist Ministry Resource C...,Carbon,0.0,0.0,1


In [34]:
df_names_fixed['Total Depletion Rights'].sum()

3771083.0

### Reference List of Owner Names from Previous Analysis

Import data from previous analysis

In [ ]:
df_ref = pd.read_excel('Water_Rights_URC_20210823.xlsx', sheet_name = 'Carbon County')[['ID', 'Water Right', 'Priority Date', 'Diversion', 'Depletion', 'Source', 'Status', 'Owner Name']].dropna(how='all')
df_ref_summary = pd.read_excel('Water_Rights_URC_20210823.xlsx', sheet_name = 'Carbon County')[['Owner Name.1', 'Total Depletion Rights', 'Total Diversion Rights', '# of Water Rights']].dropna(how='all')

In [35]:
url = 'https://www.waterrights.utah.gov/distinfo/colorado/WRPriorityDDview.asp?county='
county_name = 'CN'
''.join([url, county_name])

'https://www.waterrights.utah.gov/distinfo/colorado/WRPriorityDDview.asp?county=CN'